<a href="https://colab.research.google.com/github/Maplemx/Agently/blob/main/playground/open_source_navigator_agent-Wang_Yafang.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Open Source Navigator Agent by Wang Yafang

## Demo Description

**Author:**

[@Wang Yafang](https://github.com/yafangwang9) 王雅芳

Wang yafang major in Electronic Information Engineering, with experience in target detection systems, sentiment analysis of minority languages. She is the operation manager of the Machine Heart open-source technology community.

电子信息工程专业，做过目标检测系统、小语种情感识别，机器之心开源技术社区运营

**Showcase Description:**

OpenSource Navigator Agent is an intelligent assistant agents system I build for daily community operation and management, helping me interaction and communication with developers more efficiently.

This system have 3 major agent:

- **Tech Query Assistant**: When encountering technical issues in popular fields such as LLM, RAG, Agent, etc., it can automatically search Google and quickly find relevant reference content for the problems.

- **Open Source Highlight Miner Assistant**: It provides a GitHub open-source project link for popular open-source projects, capable of extracting the highlights of the project and organizing them into content that developers can easily understand, thereby attracting developers' attention.

- **Paper Retrieval Assistant**: In academic research, when there is a need to address specific research questions from papers, it can retrieve local paper PDFs and provide concise and clear answers.

<img width="800" alt="image" src="https://github.com/Maplemx/Agently/assets/4413155/74e9a402-e352-4e08-bfce-759d1f1ecd91">

**Introduction Video**:

Bilibili: https://www.bilibili.com/video/BV1PJ4m1v74g

## Showcase Code

### Package Installation

In [ ]:
!pip install -q -U Agently langchain beautifulsoup4 PyPDF2

### Showcase Code

In [4]:
import Agently
import requests
from bs4 import BeautifulSoup
import json
import os,json,yaml
from PyPDF2 import PdfReader
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain.chains.question_answering import load_qa_chain
from langchain.llms.base import LLM
from typing import List, Optional
from langchain_community.embeddings import JinaEmbeddings

# Settings
GLM_API_KEY = ""# add your GLM api key
GOOGLE_SEARCH_API_KEY = ""
proxies = {'https': None, 'http': None} # add your http
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36'}

# Main Code
def get_opensource_info(url):
    response = requests.get(url, headers=headers,proxies=proxies)
    if response.status_code == 200:
        readme_url = url + "/blob/main/README.md"
        readme_response = requests.get(readme_url, headers=headers, proxies=proxies)
        soup = BeautifulSoup(readme_response.content,'html.parser')
        paragraphs = soup.find_all('p')
        texts = [p.get_text() for p in paragraphs ]
        result = ' '.join(texts)
        return result
    else:
        print("开源项目信息获取失败")

def google_search(query):
    url = 'https://google-api31.p.rapidapi.com/websearch'
    headers = {
        'content-type': 'application/json',
        'X-RapidAPI-Key': GOOGLE_SEARCH_API_KEY,
        'X-RapidAPI-Host': 'google-api31.p.rapidapi.com'
    }
    data = {
        'text': query,
        'safesearch': 'off',
        'timelimit': '',
        'region': 'wt-wt',
        'max_results': 3  # 你可以根据需要调整结果的数量
    }
    response = requests.post(url, headers=headers, data=json.dumps(data))
    if response.status_code == 200:
        results = response.json()['result']
        formatted_results = ''
        for item in results:
            formatted_results = formatted_results+item['title'] +'\n'+ item['href'] +'\n'+ item['body'] +'\n'+'\n'
        return formatted_results
    else:
        return f"Error: {response.status_code}"


def process_text(text):
    # Split the text into chunks using Langchain's CharacterTextSplitter
    text_splitter = CharacterTextSplitter(
        separator="\n",
        chunk_size=1000,
        chunk_overlap=200,
        length_function=len
    )
    chunks = text_splitter.split_text(text)

    # Convert the chunks of text into embeddings to form a knowledge base
    embeddings = JinaEmbeddings(
    jina_api_key="", model_name="jina-embeddings-v2-base-en") # add jina_api_key
    knowledgeBase = FAISS.from_texts(chunks, embeddings)
    return knowledgeBase

def retrieve_pdf(query):
    pdf_reader = PdfReader('2401.03462.pdf')
    text = ""
    for page in pdf_reader.pages:
        text += page.extract_text()
        # Create the knowledge base object
    knowledgeBase = process_text(text)
    docs = knowledgeBase.similarity_search(query)
    return docs


agent_factory = Agently.AgentFactory(is_debug = False)
agent_factory\
    .set_settings("current_model", "ZhipuAI")\
    .set_settings("model.ZhipuAI.auth", { "api_key": GLM_API_KEY })


# 注册get_opensource_info到全局工具管理器
Agently.global_tool_manager.register(
    tool_name="get_opensource_info",
    desc="Get information about an open-source project from GitHub",
    args={"url": "string"},
    func=get_opensource_info
)

# 注册google_search到全局工具管理器
Agently.global_tool_manager.register(
    tool_name="google_search",
    desc="Perform a Google search and return formatted results",
    args={"query": "string"},
    func=google_search
)

# 注册get_opensource_info到全局工具管理器
Agently.global_tool_manager.register(
    tool_name="retrieve_pdf",
    desc="Get information about an open-source project from GitHub",
    args={"url": "string"},
    func=retrieve_pdf
)


# 创建各专业领域代理
tech_query_agent = (
    agent_factory.create_agent()
    .use_public_tools(["google_search"])  # 使用google_search工具
    .set_role("role", "在今后的对话中，您将作为我的技术问题智能助理，你会进行google_search，为大型语言模型（LLM）、知识检索增强RAG（Retrieval-Augmented Generation）、Agent等LLM热门应用领域的技术问题提供解决方案。")
)

content_creator_agent = (
    agent_factory.create_agent()
    .use_public_tools(["get_opensource_info"])  # 使用我们刚注册的工具
    .set_role("role", "在今后的对话中，您将作为我的人工智能写作助理，请根据我给出github链接的项目要点编写成一段摘要不要分段。在每次编写摘要时，请你参考下面的示例：阿卜杜拉国王科技大学开发的“角色扮演”（Role-Playing）的多Agent框架，可通过角色扮演提高Agent能力。CAMEL通过多个Agent之间进行对话和合作来完成分配的任务。具体来说，该框架采用两个Agent，并让Agent相互交谈，并使用启示式提示（Inception Prompt）来引导聊天Agent完成任务，并与人类意图保持一致。Agent会被赋予不同的角色，每个Agent都有自己的个性，并拥有相应角色的专业知识背景，从而利用其知识来找到满足共同任务的解决方案。")
)


paper_retrieve_agent = (
    agent_factory.create_agent()
    .set_role("role", "在今后的对话中，您将作为我的论文检索助理，能够进行检索并解答我的问题")
)


# 设置代理池
agent_pool = {
    "TechQuery": tech_query_agent,
    "ContentCreator": content_creator_agent,
    "RetrieveAssistant": paper_retrieve_agent
}

# ---------------------

# 启动演示环境
user_choice = None
while user_choice not in ("0", "1", "2"):
    user_choice = input("Choose an agent [0: TechQuery; 1: ContentCreator; 2: RetrieveAssistant]: ")

agent_names = ["TechQuery", "ContentCreator", "RetrieveAssistant"]
chosen_agent_name = agent_names[int(user_choice)]
chosen_agent = agent_pool[chosen_agent_name]
user_input = input("[User]: ")
chosen_agent.active_session()

response = (
    chosen_agent
    .user_info("An anonymous user")
    .input({"input": user_input})
    .instruct(
        "Introduce yourself and welcome user first, \
        then answer user's question {input}, \
        then tell user he/she can exit this dialogue by input 'exit' anytime."
    )
    .start()
)

print(f"[{chosen_agent_name}]: ", response)

while True:
    user_input = input("[User]: ")
    if user_input == "exit":
        break
    response = (
        chosen_agent
        .input(user_input)
        .start()
    )
    print(f"[{chosen_agent_name}]: ", response)

Choose an agent [0: TechQuery; 1: ContentCreator; 2: RetrieveAssistant]: 1
[User]: https://github.com/Maplemx/Agently
[ContentCreator]:  Agently is an AI agent development framework designed to enable developers to quickly build native applications incorporating AI agents. This framework simplifies the process of integrating AI agents into business code, allowing developers to focus on logic rather than catering to language models. The Agently framework operates on the principle of "Ask-Get Response," replacing traditional programming methods with a more natural language approach to problem-solving. Notably, Agently 3.0 introduces a plugin-to-enhance design, which simplifies the development of various agent components and allows for easier management and updates. Plugin developers can also share and install these components independently, facilitating a more modular and efficient development process. For a deeper dive into Agently, interested parties can explore the provided documentat

---

[**_<font color = "red">Agent</font><font color = "blue">ly</font>_** Framework - Speed up your AI Agent Native application development](https://github.com/Maplemx/Agently)